<a href="https://colab.research.google.com/github/UtkarshMains/decisiontree-MLP/blob/main/LSRK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar,warnings,itertools,matplotlib,keras,shutil
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from datetime import datetime
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.metrics import accuracy_score,mean_squared_error,recall_score,confusion_matrix,f1_score,roc_curve, auc
warnings.filterwarnings('ignore') 

In [2]:
df=pd.read_csv("/content/drive/MyDrive/DataCoSupplyChainDataset (1).csv",header=0,encoding='unicode_escape')

In [3]:
df['Customer Full Name'] = df['Customer Fname'].astype(str)+df['Customer Lname'].astype(str)
df.shape

(180519, 54)

In [4]:
data=df.drop(['Customer Email','Product Status','Customer Password','Customer Street','Customer Fname','Customer Lname','Product Description','Product Image','Order Zipcode','shipping date (DateOrders)'],axis=1)
data.shape

(180519, 44)

In [5]:
data['Customer Zipcode']=data['Customer Zipcode'].fillna(0)

In [6]:
data['TotalPrice'] = data['Order Item Quantity'] * data['Order Item Total']
data['order date (DateOrders)'].max()

'9/9/2017 9:50'

In [7]:
present = dt.datetime(2018,2,1)
data['order date (DateOrders)'] = pd.to_datetime(data['order date (DateOrders)'])

In [8]:
train_data=data.copy()

In [9]:
train_data['fraud'] = np.where(train_data['Order Status'] == 'SUSPECTED_FRAUD', 1, 0)
train_data['late_delivery']=np.where(train_data['Delivery Status'] == 'Late delivery', 1, 0)

In [10]:
train_data.drop(['Delivery Status','Late_delivery_risk','Order Status','order date (DateOrders)'], axis=1, inplace=True)

In [11]:
le = preprocessing.LabelEncoder()
train_data['Customer Country']  = le.fit_transform(train_data['Customer Country'])
train_data['Market']            = le.fit_transform(train_data['Market'])
train_data['Type']              = le.fit_transform(train_data['Type'])
train_data['Product Name']      = le.fit_transform(train_data['Product Name'])
train_data['Customer Segment']  = le.fit_transform(train_data['Customer Segment'])
train_data['Customer State']    = le.fit_transform(train_data['Customer State'])
train_data['Order Region']      = le.fit_transform(train_data['Order Region'])
train_data['Order City']        = le.fit_transform(train_data['Order City'])
train_data['Category Name']     = le.fit_transform(train_data['Category Name'])
train_data['Customer City']     = le.fit_transform(train_data['Customer City'])
train_data['Department Name']   = le.fit_transform(train_data['Department Name'])
train_data['Order State']       = le.fit_transform(train_data['Order State'])
train_data['Shipping Mode']     = le.fit_transform(train_data['Shipping Mode'])
train_data['Order Country']     = le.fit_transform(train_data['Order Country'])
train_data['Customer Full Name']= le.fit_transform(train_data['Customer Full Name'])
train_data.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Category Id,Category Name,Customer City,Customer Country,Customer Id,...,Order State,Product Card Id,Product Category Id,Product Name,Product Price,Shipping Mode,Customer Full Name,TotalPrice,fraud,late_delivery
0,1,3,4,91.250000,314.640015,73,40,66,1,20755,...,475,1360,73,78,327.75,3,1876,314.640015,0,0
1,3,5,4,-249.089996,311.359985,73,40,66,1,19492,...,841,1360,73,78,327.75,3,5378,311.359985,0,1
2,0,4,4,-247.779999,309.720001,73,40,452,0,19491,...,841,1360,73,78,327.75,3,4429,309.720001,0,0
3,1,3,4,22.860001,304.809998,73,40,285,0,19490,...,835,1360,73,78,327.75,3,12929,304.809998,0,0
4,2,2,4,134.210007,298.250000,73,40,66,1,19489,...,835,1360,73,78,327.75,3,10638,298.250000,0,0


In [34]:
def pred(model,cpre):
  try:
    xf=train_data.loc[:, train_data.columns != cpre]
    yf=train_data[cpre]
    xf_train, xf_test,yf_train,yf_test = train_test_split(xf,yf,test_size = 0.2,random_state = 42)
    model=model.fit(xf_train,yf_train)
    yf_pred=model.predict(xf_test)
    return accuracy_score(yf_pred, yf_test)
  except:
    return 0

In [13]:
models={}

LogisticRegression

In [14]:
lr= LogisticRegression(solver='lbfgs',random_state=0)
models['LogisticRegression']=lr

SVM

In [15]:
sv = svm.LinearSVC()
models['svm']=sv

K nearest neighbour

In [16]:
knn = KNeighborsClassifier(n_neighbors=1)
models['KNN']=knn

Random forest

In [17]:
rfc = RandomForestClassifier()
models['RandomForestClassifier']=rfc

In [18]:
train_data.columns

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Category Id',
       'Category Name', 'Customer City', 'Customer Country', 'Customer Id',
       'Customer Segment', 'Customer State', 'Customer Zipcode',
       'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market',
       'Order City', 'Order Country', 'Order Customer Id', 'Order Id',
       'Order Item Cardprod Id', 'Order Item Discount',
       'Order Item Discount Rate', 'Order Item Id', 'Order Item Product Price',
       'Order Item Profit Ratio', 'Order Item Quantity', 'Sales',
       'Order Item Total', 'Order Profit Per Order', 'Order Region',
       'Order State', 'Product Card Id', 'Product Category Id', 'Product Name',
       'Product Price', 'Shipping Mode', 'Customer Full Name', 'TotalPrice',
       'fraud', 'late_delivery'],
      dtype='object')

In [19]:
cols=['Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer','Order Item Discount',
       'Order Item Discount Rate', 'Sales','Order Profit Per Order',
       'Product Price', 'Shipping Mode',
       'fraud', 'late_delivery']

In [35]:
dic={'Columns':[]}
for i in models.keys():
  dic[i]=[]
for i in models:
  for j in cols:
    if j not in dic['Columns']:
      dic['Columns'].append(j)
    print(j)
    acc=pred(models[i],j)
    dic[i].append(acc)


Days for shipping (real)
Days for shipment (scheduled)
Benefit per order
Sales per customer
Order Item Discount
Order Item Discount Rate
Sales
Order Profit Per Order
Product Price
Shipping Mode
fraud
late_delivery
Days for shipping (real)
Days for shipment (scheduled)
Benefit per order
Sales per customer
Order Item Discount
Order Item Discount Rate
Sales
Order Profit Per Order
Product Price
Shipping Mode
fraud
late_delivery
Days for shipping (real)
Days for shipment (scheduled)
Benefit per order
Sales per customer
Order Item Discount
Order Item Discount Rate
Sales
Order Profit Per Order
Product Price
Shipping Mode
fraud
late_delivery
Days for shipping (real)
Days for shipment (scheduled)
Benefit per order
Sales per customer
Order Item Discount
Order Item Discount Rate
Sales
Order Profit Per Order
Product Price
Shipping Mode
fraud
late_delivery


In [37]:
dfs=pd.DataFrame(dic)
dfs

,Columns,LogisticRegression,svm,KNN,RandomForestClassifier
0,Days for shipping (real),0.313345,0.257146,0.847939,0.874972
1,Days for shipment (scheduled),0.597884,0.597828,0.890372,1.000000
2,Benefit per order,0.000000,0.000000,0.000000,0.000000
3,Sales per customer,0.000000,0.000000,0.000000,0.000000
4,Order Item Discount,0.000000,0.000000,0.000000,0.000000
5,Order Item Discount Rate,0.000000,0.000000,0.000000,0.000000
6,Sales,0.000000,0.000000,0.000000,0.000000
7,Order Profit Per Order,0.000000,0.000000,0.000000,0.000000
8,Product Price,0.000000,0.000000,0.000000,0.000000
9,Shipping Mode,0.597884,0.446710,0.890372,1.000000


In [38]:
dfs.to_excel('saved_file.xlsx')